In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

startTime = time.time()

fileName = "Basketball Data/Play_by_Play_New.csv"
df = pd.read_csv(fileName)

playerMatch = "Basketball Data/Player_Map.csv"
player_df = pd.read_csv(playerMatch)

teamMatch = "Basketball Data/Team_Map.csv"
player_df = pd.read_csv(teamMatch)

endTime = time.time()
print("Time Elapsed:", endTime - startTime, "sec")

Time Elapsed: 40.509000062942505 sec


In [6]:
startIndex = np.where(df["Season"] == 2016)[0][0]

print("Headers:")
for header in df:
    print(header)
    df[header] = df[header][startIndex:]
print()

Headers:
Game_id
Season
Season_Type
Game_No
Playoff_Rd
Playoff_Rd_Game_No
Date
Home_Team_id
Home_Tm
Visitor_Team_id
Away_Tm
Period
Event_Num
Wall_Clock_Time
Play_Clock_Time
Team_Committing_Action
Person1
Person2
Person3
Home_PTS
Visitor_PTS
X_Location
Y_Location
Description
Rebound_Designation
Shot_Value
Shot_Outcome
Shot_Side_of_Ct
Shot_Distance
General_Description
Player1
Player2
Player3
Player4
Player5
Player6
Player7
Player8
Player9
Player10

